# Cancer Agnostic Classification Methodology

Import all packages relevant to the workflow. These include:
- standard python packages
- sciki-rt which can be installed using the following documentation https://codeshare.phy.cam.ac.uk/hp346/scikit-rt/-/blob/master/docs/markdown/installation.md
- the classification classes that includes functionality for the classification methodology from classification_classes.py

Define paths. These include:
- the working directory where you have stored this script on your local machine 
- the data directory where you have stored the downloaded patient dataset 
- the directory of the parameter files for the elastix registration, which will be at 'x' location in the scikit-rt download 

Define global variables. These will vary dependent on the use-case and need to be hardcoded and defined explicitly by the user. 
These include:
- Dose prescriptions, specified as high dose, intermediate dose and low dose regions  
- Patient specific parameters: 
    - these include dictionaries that include the new name of the structure of interest, and the values as all the possible labels that could have been used by the clinican when delineating these patient structures. Wild cards can be used when defining the values.
    - patient list: this creates a list of the patient ids of interest where the data is stored, these are then used for each patient when running the registration and saving outputs

In [ ]:
# import standard python packages 
import os
import skrt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# import scikit-rt modules 
from skrt import Patient, ROI, StructureSet, Image
from skrt.registration import get_default_pfiles, Registration, set_elastix_dir
from skrt.better_viewer import BetterViewer
from import_analysis import ImportPatient
from skrt.simulation import SyntheticImage

# import classfication module 
from classification_classes import Patient_Info, Registration_Step, Transform_Structures, Create_Structures, Classification

### Matplotlib Plot Settings 

In [ ]:
# Set Matplotlib runtime configuration.
# For details of Matplotlib configuration, see:
# https://matplotlib.org/stable/tutorials/introductory/customizing.html
# Sizes are in points.

# figure size
plt.rc('figure', figsize=(20, 5))

# For axes, set spacing (pad) and size of label and title
plt.rc("axes", labelpad=0, labelsize=25, titlepad=17, titlesize=25)

# Set default text charactieristics.
# Possible weight values are:
# 100, 200, 300, 400 / "normal", 500, 600, 700 / "bold", 800, 900.
plt.rc("font", family="serif", serif=["Times"], size=20, weight=400)

# Set mathematics typeface when using matplotlib's built-in fonts.
plt.rc("mathtext", fontset="dejavuserif")

# Use TeX/LaTeX for typesetting.  (This requires working TeX/LaTeX installation.)
plt.rc("text", usetex=False)

# Set fontsize for legend labels.
plt.rc("legend", fontsize=12)

# For ticks, set label size and direction ("in", "out", "inout").
plt.rc(("xtick", "ytick"), labelsize=25, direction="out")

# For major and minor ticks, set size and width.
# For major ticks, set spacing (pad) of label.
plt.rc(("xtick.major"), pad=3)
plt.rc(("xtick.major", "ytick.major"), size=9, width=1.0)
plt.rc(("xtick.minor", "ytick.minor"), size=4.5, width=1.0)
plt.rc(("ytick.major"), pad=2)

# Create dictionary of default plotting options.
kwargs = {
    # Make plots interactive ("no_ui": False) or non-interactive ("no_ui": True).
    "no_ui": False,
    # Set figure size in inches.
    "figsize": (10, 6),
    # Show major ticks at specified interval (axis units).
    "major_ticks": 60,
    # Show minor ticks for specified number of intervals between major ticks.
    "minor_ticks": 5,
    # Indicate whether axis units should be mm or numbers of voxels.
    "scale_in_mm" : True,
    # Indicate whether ticks should be shown on all sides.
    "ticks_all_sides": True,
    # Specify zoom factor.
    "zoom": 2.0,
}

# Set fontsize for annotations.
# Value set via plt.rc("font", size=20) seems not to be applied consistently.
#annotation_fontsize = 10

# Define Trial Specific Variables
- Each patient cohort will have their own variable set-up including the following variables 
    - trial name 
    - dose prescriptions ot each risk region 
    - dictionaries for the naming schemes of each structure based on clinican labelling

In [ ]:
## Global Variables 
# define trial or dataset name
trial = '<trial_name>' 

# define [low_risk, intermediate_risk, high_risk] dose prescriptions 
# any dose values can be left as 0 and entire arms can be marked as [0,0,0]
control_arm = [54, 60, 65] # the trials most common treatment route 
trial_arm_one = [0, 54, 60] # the trials second most common route
trial_arm_two = [0, 0, 50] # the trials comorbidity route 
trial_arm_dose_prescriptions = [control_arm, trial_arm_one, trial_arm_two]

# define structure dictionaries for recurrence names, CTV, and comparison structure names 
recurrence_dict = {'tumour_volume':['Rec_CT _LN', 'Reccurence_MRI', 'rec_MRI',
                    '*resid*','*recurrence*', '*LR*', '*lr*', 
                    'Rec_CT', 'Rec_PET', 'rec_PETCT','REC-CT', 
                    'Reccurence', 'Recurrence CT', 'Rec_CT_primary', 
                    'relapse', 'relapse_GCB', 'L nodal rec', 'Rec_PET-CT', 
                    'Recurrence_RS'], 'thyroid':['*thy*', '*TC*', '*tc*', 'tc_REC']}

# define comparison structure dictionary: example here is given as the thyroid cartilage 
comparison_struct_dict = {'thyroid':['*thy*', '*TC*', '*tc*']}

# define ctv structure dictionary: example names of possible CTV structure naming schemes and definitions 
ctv_names = {'CTV_TB': ['*CTV*65*', '*CTV*60*', '*CTV*54*', '*CTV*55*', '*CTV*50*', 'r cvt60']} 
comparison_metrics_list = []

## Global Paths 
# set up paths to working directory for this script 
working_dir = '<insert directory containing directoy with patient data>'

# set up paths for registration parameter files 
pfile_dir = "<path for the directory containing the parameter files in the elastix directory>"
elastix_dir = "<path for the elastix directory as part of the scikit-rt package>"
output_data_dir = f'{working_dir}/data/03_primary/{trial}'

# patient list 
patient_list = os.listdir(data_dir)


## Primary Workflow
This code loops through each patient in your dataset directory and executes the following:
1. uses the Registration Class to execute the elastix registration workflow using the assigned fixed and moving images 
2. uses the Transform_Structures Class to transform the structures assigned to the relapse CT scan using the deformation vector field obtained from the Registration step 
3. uses the Create Structures class to generate the D95% structures using the treatment planning dose field, and generate the spheroid structure around the centroid of the mapped relapse gross tumour (rGTV)
4. uses the Classification Class to determine the location of the mapped centroid rGTV and the volume of the relapse spheroid relative to the D95% structures, to determine the classification 
5. uses scikit-rt functionality to obtain comparison metrics between the mapped rGTV and planning structures 

In [ ]:
#define empty dictionary for storing metrics 
metrics_df = pd.DataFrame()

# loop through each patient to perform the classification 
for patient in patient_list: 

    # define each patient as a Patient Object for relevant data structures 
    if trial == 'import': 
        p = ImportPatient(f'{data_dir}/{patient}')
    else:
        p = Patient(f'{data_dir}/{patient}')

    # define directory for registration results 
    results_dir = f'results/planning_fixed/{trial}/{p.id}'
    
    # if you want to test some of the code put the results here 
    #results_dir = f'tester/planning_fixed/{trial}/{p.id}'
    print('> Patient:', p.id)

    # register matching fixed and moving patient scans 
    registration = Registration_Step(p, trial, recurrence_dict, ctv_names, 
                                        comparison_struct_dict, pfile_dir, results_dir)
    adjusted_fixed = registration.adjusted_fixed
    registration.perform_reg()

    # instantiate Transform Structures Object 
    structure_transform = Transform_Structures(p, trial, recurrence_dict, ctv_names, comparison_struct_dict, results_dir)
    
    # write structures to results directory 
    ctv = structure_transform.write_structures()
    print('> Structures Identified and CTV ROI isolated and writtten to file')

    # transform the relapse structures using the registration parameters
    relapse_ss_transform, relapse_tv_transform_roi, registration_comparison_struct_metrics = structure_transform.reg_structures(ctv)
    print('> Relapse structures transformed')
    
    # create CTV structure set
    ctv_ss = StructureSet(image=adjusted_fixed)
    ctv_ss.add_roi(ctv)
    print('> CTV isolated and added to independent Structure Set')
    
    # create relapse sphere with radius set to the global uncertainty for the patient cohort 
    relapse_sphere = Create_Structures(p, trial, recurrence_dict, ctv_names, comparison_struct_dict).create_structure_spheroid(roi=relapse_tv_transform_roi, im='adjusted_fixed', radius=5, name='relapse_sphere')
    print('> Synthetic Relapse Sphere created')
   
    # create structure set with regions containing 95% of dose prescriptions
    ss, trial_arm_dose_prescriptions_95 = Create_Structures(p, trial, recurrence_dict, ctv_names, comparison_struct_dict).create_isodose_structures(trial_arm_dose_prescriptions)
    print('> Isodose 95% Structure created')
    
    # compare the relevant comparison structures to obtain local uncertainty  
    comparison_ss = registration.comparison_ss
    comparison_metrics = comparison_ss.get_comparison(relapse_ss_transform, metrics=['abs_centroid'])
    print('> ROI Comparison:', comparison_metrics)
    #comparison_metrics_list.append(comparison_metrics)

    
    # obtain the trial arm or protocol type of the patient (these are relevant to the provide dose prescriptions)
    trial_arm = Classification(p, trial, recurrence_dict, ctv_names, comparison_struct_dict, results_dir).get_trial_arm(trial_arm_dose_prescriptions, ctv)
    print('> Trial Arm or Treatment Route Determined')

    # get the centroid classification 
    dose_dict, sim_sphere_centroid_dose, centroid_dose_class_dict = Classification(p, trial, recurrence_dict, ctv_names, comparison_struct_dict, results_dir).voxtox_centroid_dose_classification(sim_roi_sphere=relapse_sphere, isodose_ss=ss, trial_arm=trial_arm, trial_arm_dose_prescriptions=trial_arm_dose_prescriptions_95)
    print('> Centroid Classification Determined')
    
    # get the volume classification 
    volume_class_dict = Classification(p, trial, recurrence_dict, ctv_names, comparison_struct_dict, results_dir).voxtox_volume_dose_classification(sim_roi_sphere=relapse_sphere, dose_dict=dose_dict, trial_arm=trial_arm, trial_arm_dose_prescriptions=trial_arm_dose_prescriptions_95)
    print('> Volume Classification Determined')

    # print results of classification and protocol type/trial arm 
    print('> Trial Arm:', trial_arm)
    print('> Centroid Classification Dict:', centroid_dose_class_dict)
    print('> Volume Classification Dict:', volume_class_dict)

    
    # visualise fixed image with transformed and original structures, and planning treatment dose field  
    adjusted_fixed.view(rois=[ss, relapse_sphere, ctv_ss], dose=registration.dose, legend=True, colorbar=2)

    # Obtain Metrics and Save Patient Information 
    classification_dict = {}
    classification_dict['centroid classification'] = centroid_dose_class_dict
    classification_dict['volume classification'] = volume_class_dict
    classification_df = pd.DataFrame.from_dict(classification_dict)
    classification_df['patient'] = p.id
    classification_df['trial_arm'] = trial_arm
    metrics_df = pd.concat([metrics_df, classification_df])

    # save results for all patients to trial results folder
    if not os.path.exists(f"{output_data_dir}/{p.id}"):
        os.mkdir(f"{output_data_dir}/{p.id}")
    classification_df.to_csv(f'{output_data_dir}/{patient}/{patient}_classifications.csv')
    metrics_df.to_csv(f'{output_data_dir}/{trial}_all_classifications.csv')
    

    # set outdir as the patient specific directory 
    outdir = f"{output_data_dir}/{p.id}"
    if not os.path.exists(outdir):
        os.mkdir(outdir)

    # create df with the comparison data for each indiviudal patient 
    ctv_comparison_metrics_df = pd.concat([total_vol_comparison, spheroid_vol_comparison])
    ctv_comparison_metrics_df['patient'] = p.id"""
    

## Final Visualisation
- Visualise the original planning image, with relabelled and transformed structures 

In [ ]:
# visualise fixed image with transformed and original structures, and planning treatment dose field  

roi_names={
    'D95% Tumour Bed':'isodose_high_risk_a1',
    'D95% Whole Breast':'isodose_low_risk_a1', 
    'D95% Partial Breast':'isodose_intermediate_risk_a1', 
    'Clinical Tumour Bed Volume':'CTV_TB',
    'Recurrence Sphere':'relapse_sphere',
    'rGTV':'tumour_volume'}

adjusted_fixed.view(rois=[ss, relapse_sphere, ctv_ss, relapse_ss_transform], 
                        dose=p.get_dose_sum(), legend=True, colorbar=2, 
                        roi_names=roi_names, 
                        roi_colors={'rGTV':'magenta', 'Recurrence Sphere':'pink',
                                     'Clinical Tumour Bed Volume':'red'})